In [1]:
import os
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Column, MaskedColumn, vstack
from astropy.nddata import Cutout2D
from astropy.wcs import WCS

import sys
sys.path.append('/melkor/d1/guenther/soft/python/psfsubtraction/')
import psfsubtraction.prepare.centering

%matplotlib inline

In [2]:
import ipyparallel

rc = ipyparallel.Client()
dview = rc[:]
lview = rc.load_balanced_view()

In [3]:
datadir = '/melkor/d1/guenther/obs/HST/Cepheids/snapshotprogram/'
outdir = '/melkor/d1/guenther/projects/Cepheids/HSTsnapshot/'

In [4]:
fitslist = glob(os.path.join(datadir, '?????????_drz.fits'))

F621Mfiles = np.array([f for f in fitslist if (fits.getval(f, 'FILTER') == 'F621M')])
F621Mnames = [fits.getval(f, 'TARGNAME') for f in F621Mfiles]
F621Mfiles = F621Mfiles[np.argsort(F621Mnames)]


F845Mfiles = np.array([f for f in fitslist if (fits.getval(f, 'FILTER') == 'F845M')])
F845Mnames = [fits.getval(f, 'TARGNAME') for f in F845Mfiles]
F845Mfiles = F845Mfiles[np.argsort(F845Mnames)]

In [5]:
def read_images(filelist, halfwidth):
    '''Read files, make header wrappers for WCS'''
    images = np.ma.zeros((2 * halfwidth + 1, 2 * halfwidth + 1, len(filelist)))
    targets = []

    for i, f in enumerate(filelist):
        hdus = fits.open(f)
        xm, ym = psfsubtraction.prepare.centering.center_from_spikes(hdus[1].data)
        targets.append(Cutout2D(hdus[1].data, (xm, ym), halfwidth, wcs=WCS(hdus[1].header)))
        xm = np.int(xm + 0.5)
        ym = np.int(ym + 0.5)
        images[:, :, i] = hdus[1].data[xm - halfwidth: xm+halfwidth + 1,
                                       ym - halfwidth: ym+halfwidth + 1]
        hdus.close()
    return images, targets


In [6]:
F621Marr, F621M = read_images(F621Mfiles, 50)
F845Marr, F845M = read_images(F845Mfiles, 50)

In [7]:
def mask_saturated(images, maskingfunc):
    '''Mask pixels above a certain value
   
    Parameters
    ----------
    image : np.array
        2d or 3d array of images (if 2d, then array of flattend images)
    maskingfunc : callable
        This function is called for every image. It returns as boolean mask
        that is ``True`` for all values that should be masked.
    '''
    for i in range(images.shape[-1]):
        images[maskingfunc(images[..., i]), i] = np.ma.masked
    return images

In [8]:
F621Marr = mask_saturated(F621Marr, lambda x: x > 0.6 * x.max())
F845Marr = mask_saturated(F845Marr, lambda x: x > 0.6 * x.max())

In [9]:
normF621 = np.ma.median(F621Marr.reshape((-1, F621Marr.shape[-1])), axis=0)
normF845 = np.ma.median(F845Marr.reshape((-1, F845Marr.shape[-1])), axis=0)

medianimF621 = np.ma.median(F621Marr.reshape((-1, F621Marr.shape[-1])), axis=1)
medianimF621 /= np.ma.median(medianimF621)
medianimF845 = np.ma.median(F845Marr.reshape((-1, F845Marr.shape[-1])), axis=1)
medianimF845 /= np.ma.median(medianimF845)

In [10]:
def apply_mediannorm(arr, normperim, normim):
    shape = arr.shape
    arr = arr.reshape((-1, shape[-1]))
    arr /= normperim[None, :]
    arr /= normim[:, None]
    return arr.reshape(shape)

def remove_mediannorm(arr, normperim, normim):
    shape = arr.shape
    arr = arr.reshape((-1, shape[-1]))
    arr *= normperim[None, :]
    arr *= normim[:, None]
    return arr.reshape(shape)

In [11]:
normF621 = apply_mediannorm(F621Marr.copy(), normF621, medianimF621)
normF845 = apply_mediannorm(F845Marr.copy(), normF845, medianimF845)

In [20]:
F621_stricter_masking = mask_saturated(F621Marr, lambda x: x > 0.2 * x.max())
F621_base = np.ma.array(normF621, mask=F621_stricter_masking, copy=True)

from psfsubtraction.fitpsftemplates.fitters import UseAllPixelsSubtraction as UAPS

F621inputs = [UAPS(normF621[..., i], np.delete(F621_base, i, axis=-1)) for i in range(F621_base.shape[-1])]

In [21]:
@lview.parallel
def psffit_single_image(psffitter):
    return psffitter.remove_psf()

In [23]:
asyncres = psffit_single_image.map(F621inputs)

AttributeError: 'function' object has no attribute 'map'

In [ ]:
# Vegamag zero points are here http://www.stsci.edu/hst/wfc3/phot_zp_lbn

# Units in image are electrons/s
# Images are drz -> drizzeled -> pixels are all same size.


def flux2magF621M(x):
    return -2.5 * np.log10(x) + 24.4539


def flux2magF845M(x):
    return -2.5 * np.log10(x) + 23.2809

In [ ]:







halfwidth = 50
daofindkwargs = {'fwhm': 1.5, 'threshold': 7, 'roundlo': -0.8, 'roundhi': 0.8}

fluxes, imout, scaledout = detection.photometryloop(images, targets, **daofindkwargs)
fluxes.add_column(MaskedColumn(flux2magF621M(fluxes['flux_fit']), 'mag_fit'))
fluxes.add_column(MaskedColumn(['F621M'] * len(fluxes), 'filter'))
#detection.plot_gallery('PSF subtr. - linear scale', imout, 10, 7, sources=fluxes)
#detection.plot_gallery('PSF subtr. - funny scale', scaledout, 10, 7, sources=fluxes)
# Get rid of negative fluxes. They must be fit artifacts
# Investigate later where they come from.
fluxes = fluxes[~fluxes['mag_fit'].mask]